# Importar as bibliotecas necessárias.

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta.tables import *
from delta import *
import pyspark.sql.functions as PSF
import psycopg2
import pandas as pd
import os
import time

## Coletar as variaveis de ambiente definidas em [Variáveis de Ambiente](https://github.com/Renabouj/Tendencias-e-Desafios-em-SI-2021.2/blob/main/README.md#vari%C3%A1veis-de-ambiente).

In [5]:
env_vars = !cat ../Environment/.env
for var in env_vars:
    key, value = var.split('=')
    os.environ[key] = value

## Configuração do Ambiente

In [6]:
builder = pyspark.sql.SparkSession.builder.appName("PySpark_Postgres_test").config("spark.jars", os.environ.get('SPARKJAR')) \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .master("local").appName("PySpark_Postgres_test")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

22/05/18 13:03:54 WARN Utils: Your hostname, MacBook-Pro-de-Renato.local resolves to a loopback address: 127.0.0.1; using 10.0.0.102 instead (on interface en0)
22/05/18 13:03:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/renabouj/Codes/tendenciasSI/env/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/renabouj/.ivy2/cache
The jars for the packages stored in: /Users/renabouj/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e8023ca6-0b9c-4c06-8e33-ae8d3f80539f;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.2.1 in central
	found io.delta#delta-storage;1.2.1 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 185ms :: artifacts dl 23ms
	:: modules in use:
	io.delta#delta-core_2.12;1.2.1 from central in [default]
	io.delta#delta-storage;1.2.1 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnl

## Ler o arquivo a fim de enviá-lo para o banco de dados.

In [8]:
dataf = spark.read.format("csv").option("delimiter", ",").option("header", "true").option("inferSchema", "true") \
        .load("../data/200ksales.csv")
dataf.show()

+--------------------+--------------------+-------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|              Region|             Country|    Item_Type|Sales_Channel|Order_Priority|Order_Date| Order_ID| Ship_Date|Units_Sold|Unit_Price|Unit_Cost|Total_Revenue|Total_Cost|Total_Profit|
+--------------------+--------------------+-------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|Australia and Oce...|           Australia|         Meat|       Online|             C|  4/4/2011|451691138| 5/23/2011|      4300|       421|      364|      1814127|   1568167|      245960|
|                Asia|          Tajikistan|Personal Care|       Online|             L| 7/12/2018|144177377|  8/1/2018|      4145|        81|       56|       338770|    234897|      103873|
|  Sub-Saharan Africa|          Mozambique|    Cosmetic

In [9]:
lista_de_colunas_inteiro = ["Units_Sold", "Unit_Price", "Unit_Cost", "Total_Revenue", "Total_Cost", "Total_Profit"]

def convert_data_types(df, columns_list):
    for column_name in columns_list:
        df.select(PSF.col(column_name).cast('bigint').alias(column_name))
    return df

In [10]:
convert_data_types(dataf, lista_de_colunas_inteiro)

DataFrame[Region: string, Country: string, Item_Type: string, Sales_Channel: string, Order_Priority: string, Order_Date: string, Order_ID: int, Ship_Date: string, Units_Sold: int, Unit_Price: int, Unit_Cost: int, Total_Revenue: int, Total_Cost: int, Total_Profit: int]

## Salvar o arquivo 200ksales.csv no banco de dados. 

In [11]:
# O parâmetro OVERWRITE somente será True se já
# existir uma tabela cujo nome é o passado no 
# parâmetro DBNAME.

def save_data_to_database(df, dbname: str, overwrite: bool):
    if overwrite:
        df.write.format("jdbc").option("driver", "org.postgresql.Driver").option("url", os.environ.get('DB_CONNECTION_STRING')) \
        .option("dbtable", dbname).option("user", os.environ.get('USER')).option("password", os.environ.get('PASSWORD')) \
        .mode("overwrite").save()
        
        return print("Overwrite done!")
    
    else:
        df.write.format("jdbc").option("driver", "org.postgresql.Driver").option("url", os.environ.get('DB_CONNECTION_STRING')) \
        .option("dbtable", dbname).option("user", os.environ.get('USER')).option("password", os.environ.get('PASSWORD')).save()
        
        return print("Saved without overwrite!")
        
save_data_to_database(dataf, "sales", True)

Overwrite done!


ERROR:root:Exception while sending command.                                     
Traceback (most recent call last):
  File "/Users/renabouj/Codes/tendenciasSI/env/lib/python3.9/site-packages/py4j/clientserver.py", line 480, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/renabouj/Codes/tendenciasSI/env/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/renabouj/Codes/tendenciasSI/env/lib/python3.9/site-packages/py4j/clientserver.py", line 503, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
